# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import  GridSearchCV
import numpy as np
from sklearn.metrics import confusion_matrix
import pickle


In [25]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM TableName1", engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [26]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    lemmatized = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [29]:
def performance(model, X_test, y_test):
    y_predicted = model.predict(X_test)
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(y_test[col], y_predicted[:, i]))
performance(pipeline, X_test, Y_test)

related
             precision    recall  f1-score   support

          0       0.62      0.35      0.44      1559
          1       0.82      0.93      0.87      4995

avg / total       0.77      0.79      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5412
          1       0.86      0.39      0.53      1142

avg / total       0.88      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3794
          1       0.75      0.53      0.62      2760

avg / total       0.73      0.72      0.71      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
parameters = { 'vect__max_df': (0.75, 1.0),
                'clf__estimator__n_estimators': [10, 25],
                'clf__estimator__min_samples_split': [2, 6]
              }

cv = GridSearchCV (pipeline, param_grid= parameters )

cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.75, 1.0), 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.75, 1.0), 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
y_pred = cv.predict(X_test)

In [33]:
labels = np.unique(y_pred)
accuracy = (y_pred == Y_test).mean()

print("Labels:", labels)
print("Accuracy:", accuracy)
print("\nBest Parameters:", cv.best_params_)

Labels: [0 1]
Accuracy: related                   0.800580
request                   0.893653
offer                     0.996491
aid_related               0.753128
medical_help              0.920049
medical_products          0.949191
search_and_rescue         0.970857
security                  0.978639
military                  0.966128
child_alone               1.000000
water                     0.953616
food                      0.931645
shelter                   0.931950
clothing                  0.988099
money                     0.979097
missing_people            0.989014
refugees                  0.966738
death                     0.954684
other_aid                 0.871224
infrastructure_related    0.935764
transport                 0.957736
buildings                 0.951785
electricity               0.981843
tools                     0.993287
hospitals                 0.989167
shops                     0.995728
aid_centers               0.988709
other_infrastructure      0.955

In [34]:
performance(cv, X_test, Y_test)

related
             precision    recall  f1-score   support

          0       0.70      0.28      0.40      1559
          1       0.81      0.96      0.88      4995

avg / total       0.79      0.80      0.77      6554

request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5412
          1       0.87      0.46      0.60      1142

avg / total       0.89      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3794
          1       0.75      0.62      0.68      2760

avg / total       0.75      0.75      0.75      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [35]:
pickle.dump(cv, open('disastermodel.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.